In [1]:
import sys, os
path2add = os.path.normpath(os.path.abspath(os.path.join(os.path.curdir, os.path.pardir, 'TestBeam')))
if (not (path2add in sys.path)) :
    sys.path.append(path2add)

from beamtest_analysis_helper import DecodeBinary
from natsort import natsorted
from pathlib import Path
from tqdm import tqdm

import matplotlib.pyplot as plt
import json
# import statistics
import numpy as np

# translate

In [ ]:
for k in range(5,6):
    path_to_loop_dir = f"D:/fermilab/ETROC2/May11SEU/Run_May11SEU_{k+1}/loop_0" # Change this path
    files = natsorted(list(Path(path_to_loop_dir).glob('T*bin')))
    # files = files[1:] # Ignore first binary file
    print(files)
    length = len(files)
    board_ids = 0x17f0f
    write_file_cnt = 0
    write_file_name = f'Run{k}/test_{write_file_cnt}.nem'
    write_line_cnt = 0

    if not os.path.exists(f'Run{k}'):
        os.makedirs(f'Run{k}')
    for i in tqdm(range(length)):
        with open(files[i],'rb') as file:
            offset = 0
            readsize = 4
            filesize = file.read()
            enable_channel = 0
            current_type = 0 
            current_channel = 0
            words_cnt=0
            print(len(filesize))
            while offset < len(filesize):
                # print(f'offset={offset}')
                
                if (offset%4 == 0) & (readsize==4):
                    data = filesize[offset:offset+readsize]
                    u8_list = [int.from_bytes(data[z:(z+1)], byteorder='big') for z in range(readsize)]
                    u8_list=u8_list[::-1]
                    
                else:
                    first_step = 4-offset%4
                    # print(f'firts={first_step}')
                    data =  filesize[offset-5+2*first_step:offset-5+first_step:-1] + filesize[offset+first_step+3:offset+2*first_step-readsize:-1]
                    u8_list = [int.from_bytes(data[z:(z+1)], byteorder='big') for z in range(readsize)]
                    # print(u8_list)

                
                # print(readsize)
                word32 = 0
                for j, u8 in enumerate(u8_list):
                    word32 |= u8 << (8 * (readsize -1 - j))
                    # print(word32>>20)
                # word32 = int(word32)
                # print(f'word32={hex(word32>>20)}')
                
                # print(current_type)

                if (word32>>20 & 0xfff  == 0x559) & (readsize == 4):     #CLOCK
                    offset = offset + 4
                    # print('clock')
                    with open(write_file_name,'a') as output:
                        output.write(f"CLOCK {(word32%2**20):020b} \n")
                        write_line_cnt = write_line_cnt +1

                elif (word32>>20 & 0xfff == 0x556) & (readsize == 4):   #FILLER
                    offset = offset + 4
                    # print('filler')
                    # with open(write_file_name,'a') as output:
                    #     output.write(f"FILLER {hex(word32)[2:]} \n")
                    #     write_line_cnt = write_line_cnt +1 

                elif (word32>>4 & 0xfffffff == 0xc3a3c3a) & (readsize == 4):    #header
                    # print('header')
                    offset = offset + 4
                    enable_channel = word32 & 0b1111
                    current_type = 1
                    readsize = 4

                elif current_type == 1:             #'EH'
                    offset = offset + 4
                    # print('EH')
                    readsize = 5
                    event_type         = (word32) & 0x3
                    event_number       = (word32 >> 12) & 0xffff
                    words_in_event     = (word32 >> 2) & 0x3ff
                    version            = (word32 >> 28) & 0xf
                    current_type = 2
                    with open(write_file_name,'a') as output:
                        output.write(f"EH {version:04b} {event_number} {words_in_event} {event_type:02b}\n")
                        write_line_cnt = write_line_cnt +1 
                    words_cnt = words_in_event
                    # print(words_cnt)

                elif (current_type ==2) & (word32>>22 & 0x3ffff == 0x3c5c0>>2):           #H
                    # print('H')
                    offset = offset + 5
                    bcid = (word32 & 0xfff)
                    l1acounter = ((word32 >> 14) & 0xff)
                    in_40bit = True
                    Type = (word32 >> 12) & 0x3
                    current_channel = current_channel + 1
                    with open(write_file_name,'a') as output:
                        output.write(f"H {current_channel} {l1acounter} {Type:02b} {bcid}\n")
                        write_line_cnt = write_line_cnt +1 
                    current_type = 3
                    words_cnt = words_cnt - 1    

                elif (word32>>39 & 0b1 == 0b1) & (current_type == 3):
                    # print('D')
                    offset = offset + 5
                    EA = (word32 >> 37) & 0b11
                    ROW = (word32 >> 29) & 0b1111
                    COL = (word32 >> 33) & 0b1111
                    TOA = (word32 >> 19) & 0x3ff
                    TOT = (word32 >> 10) & 0x1ff
                    CAL = (word32) & 0x3ff
                    with open(write_file_name,'a') as output:
                        output.write(f"D {current_channel} {EA:02b} {ROW} {COL} {TOA} {TOT} {CAL}\n")
                        write_line_cnt = write_line_cnt +1 
                    words_cnt = words_cnt - 1

                elif (word32 >> 22) & 0x3ffff == board_ids:
                    # print('T')
                    offset = offset + 5
                    hits   = (word32 >> 8) & 0xff
                    status = (word32 >> 16) & 0x3f
                    CRC    = (word32) & 0xff
                    with open(write_file_name,'a') as output:
                        output.write(f"T {current_channel} {status} {hits} {CRC:08b}\n")
                        write_line_cnt = write_line_cnt +1 
                    words_cnt = words_cnt - 1
                    current_type = 2
                    if words_cnt == 0:
                        current_type = 4
                        readsize =4
                    
                
                elif ((word32>>26) & 0X3f == 0b001011) &  (current_type == 4):
                    # print('ET')
                    with open(write_file_name,'a') as output:
                        output.write(f"ET {event_number}\n")
                        write_line_cnt = write_line_cnt +1 
                    current_type = 0
                    current_channel = 0
                    offset = offset + 4
                    if write_line_cnt > 50000:
                        write_file_cnt = write_file_cnt +1
                        write_line_cnt = 0
                        write_file_name = f'Run{k}/test_{write_file_cnt}.nem'
                        if not os.path.exists(f'Run{k}'):
                            os.makedirs(f'Run{k}')
                        print('newfile')
                
        


                else:
                    offset = offset + 4
                    print('else')
                    current_type = 0
                    print(hex(word32))
                    # break

                # elif current_type ==3 & word32>>39 == 0b1:

        


# Trigger rate

In [ ]:
p_col=[8,8,2,2]
p_row=[0,2,0,2]
# p_col=[0,0,1,1]
# p_row=[0,1,0,1]
BCID_LIST = [719,2863,32]
pixel_list = list(zip(p_row,p_col))
print(pixel_list)
toa_1 = []
tot_1 = []
cal_1 = []
toa_2 = []
tot_2 = []
cal_2 = []
toa_3 = []
tot_3 = []
cal_3 = []
toa_4 = []
tot_4 = []
cal_4 = []
for k in range(15,16):    
    # path_to_loop_dir = f"F:/4ft" # Change this path
    # path_to_loop_dir = f"D:/fermilab/ETROC2/May11SEU/Run_May11SEU_{k}/loop_0" # Change this path
    path_to_loop_dir = f"D:/fermilab\ETROC2\ETROC-Analysis\SEU\Run{k-1}"
    files = natsorted(list(Path(path_to_loop_dir).glob('T*.nem')))
    # files = files[3:] # Ignore first 3 nem file
    eventnumber=[]
    resetcnt=[0]
    errorcode=[0000]
    trigger_rate = []
    BCID =[] 
    time = [0]
    time_reset = [0]
    T = [0]
    len(files)
    length = len(files)
    clockcnt = 0
    datacnt = 0
    linkstatus = [0]
    sd_datalength = []  
    for i in tqdm(range(length)):
        # if i % 10 == 0:
  
        with open(files[i], 'r') as file:
            lines = file.readlines()
            for line in lines:
                words = line.split()  
                if words[0]=='CLOCK':
                    clockcnt = clockcnt + 1
                    T = T + [T[-1]+0.025*1.024*1.024]
                    if [int(words[1]) & 0xff] != resetcnt[-1]:
                        resetcnt = resetcnt + [int(words[1]) & 0xff]
                        
                        time_reset = time_reset + [T[-1]]
                        # print(words[1][4:12])
                    linkstatus = linkstatus + [int(words[1][4])*8+int(words[1][6])*4+int(words[1][8])*2+int(words[1][10])]
                    errorcode = errorcode + [words[1][0:4]]
                elif words[0] == 'ET':
                    eventnumber.extend([int(words[1])])
                elif words[0] == 'H':
                    BCID.extend([int(words[4])])
                elif words[0] == 'D':
                    # print(words)
                    if (words[3]=='0') & (words[4] == '8'):
                        datacnt = datacnt + 1
                        toa_1.extend([int(words[5])])
                        tot_1.extend([int(words[6])])
                        cal_1.extend([int(words[7])])
                    elif (words[3]=='2') & (words[4] == '8'):
                        toa_2.extend([int(words[5])])
                        tot_2.extend([int(words[6])])
                        cal_2.extend([int(words[7])])
                    elif (words[3]=='0') & (words[4] == '2'):
                        toa_3.extend([int(words[5])])
                        tot_3.extend([int(words[6])])
                        cal_3.extend([int(words[7])])
                    elif (words[3]=='2') & (words[4] == '2'):
                        toa_4.extend([int(words[5])])
                        tot_4.extend([int(words[6])])
                        cal_4.extend([int(words[7])])
                if clockcnt>100:
                    # print(datacnt)
                    # print(clockcnt)
                    trigger_rate = trigger_rate + [(datacnt/clockcnt)*2048*40/1.024/1.024/4]
                    time = time + [time[-1]+clockcnt*0.025*1.024*1.024]
                    datacnt = 0
                    clockcnt = 0
    # print(trigger_rate)
    fig, ax1 = plt.subplots(figsize=(10, 6))
    color = 'tab:blue'
    ax1.set_xlabel('Time [s]', fontsize=15)
    ax1.set_ylabel('Triggre rate [Hz]', color=color, fontsize=15)
    ax1.plot(time[1:],trigger_rate[:],marker='o')
    ax1.tick_params(axis='y', labelcolor=color)

# 创建共享X轴的第二个Y轴
    ax2 = ax1.twinx()  
    color = 'tab:red'
    ax2.set_ylabel('resetcnt', color=color, fontsize=15)
    ax2.plot(time_reset, resetcnt,color=color,alpha=0.5)
    ax2.tick_params(axis='y', labelcolor=color)
    plt.title(f'Trigger rate & Resetcnt vs Time @ run{k}', fontsize=20)
    # plt.plot(time[1:],resetcnt[:],marker='o')
    # # plt.axis([0, 500, 750000,754000])
    # plt.xlabel('Time [s]')
    # plt.ylabel('Triggre rate [Hz]')
    # plt.title(f'Trigger rate on SEU Run{k}')
    # plt.show()

    fig, ax1 = plt.subplots(figsize=(10, 6))
    color = 'tab:blue'
    ax1.set_xlabel('Time [s]', fontsize=15)
    ax1.set_ylabel('linkstatus [Hz]', color=color, fontsize=15)
    ax1.plot(T[1:],linkstatus[1:])
    ax1.tick_params(axis='y', labelcolor=color)

# 创建共享X轴的第二个Y轴
    ax2 = ax1.twinx()  
    color = 'tab:red'
    ax2.set_ylabel('errorcode', color=color, fontsize=15)
    ax2.plot(T[1:], errorcode[1:],color=color,alpha=0.5)
    ax2.tick_params(axis='y', labelcolor=color)
    plt.title(f'linkstatus vs error code @ run{k}', fontsize=20)
    
    # plt.subplots()
    # plt.plot(T,linkstatus)
    # plt.xlabel('Time [s]')
    # plt.ylabel('linkstatus')
    # plt.title(f'linkstatus in run_{k}')
